In [ ]:
import gurobipy as gp
from gurobipy import GRB

m = gp.Model("KitchenFactory")

kitchen = ['A', 'B', 'C']
departments = ['taglio', 'verniciatura', 'montaggio']

daily_prod_costs = {'A': 1500, 'B': 2500, 'C': 2000}

selling_prices = {'A': 4000, 'B': 7500, 'C': 5000}


phone_costs = 150
electricity_costs = 125
daily_costs = phone_costs+electricity_costs

max_wood = 800

wood_usage = {'A': 24, 'B': 27, 'C': 23}



times = {('A', 'taglio'): 10, ('B', 'taglio'): 30, ('C', 'taglio'): 25,
         ('A', 'verniciatura'): 10, ('B', 'verniciatura'): 15, ('C', 'verniciatura'): 10,
         ('A', 'montaggio'): 8, ('B', 'montaggio'): 12, ('C', 'montaggio'): 15}

max_tme = {'taglio': 20 * 60, 'verniciatura': 18 * 60, 'montaggio': 22 * 60}

x = m.addVars(kitchen, name='x', vtype=GRB.INTEGER)


m.setObjective((sum(x[j] * (selling_prices[j] - daily_prod_costs[j]) for j in kitchen) - daily_costs), GRB.MAXIMIZE)

m.addConstr(x['A'] >= 4)
m.addConstr(x['B'] >= 5)
m.addConstr(x['C'] >= 6)

m.addConstrs((gp.quicksum(x[r] * times[r, m] for r in kitchen) <= max_tme[m] for m in departments), name='workTime')

m.addConstr(gp.quicksum(x[j] * wood_usage[j] for j in kitchen ) <= max_wood, name='woodUsage')

m.optimize()

if m.status == GRB.Status.OPTIMAL:
    print('Optimal solution:')
    for r in kitchen:
        print(f"{r}: {x[r].X:.0f}")
    print(f"Profit: {m.ObjVal:.2f}")
